<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 2 : Modularité</h1>

Le développement d'un grand porgramme demande une certaine organisation et, en particulier, un découpage des différents aspects du programme et des différentes tâches qui doivent être accomplies. Ceci est d'autant plus vrai lorsque plusieurs personnes participent au développement.  
Parmi ces questions relevent du **génie logiciel** et s'interessent à la manière dont les différentes parties d'un programme peuvent s'articuler.  
L'un des objectifs consiste à spécifier le rôle de chaque partie suffisamment précisément pour que chacune puisse ensuite être réalisé indépendamment des autres.

### Les ensembles
Le [paradoxe des anniversaires](https://images.math.cnrs.fr/Coincidences.html) nous permet de vérifier que dans un groupe (de, au moins, 23 personnes), il y a plus d'une chance sur deux pour que deux personnes aient leur anniversaire le même jour.  
On peut utiliser la fonction suivante avec des tableaux de 23 éléments (aléatoires) pour vérifier si un tableau contient un élément en double.

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = set()
    for elt in tab:
        if elt in ens:
            return True
        ens.add(elt)
    return False

On passe les élements en revue et on les stocke, à la volée, dans un ensemble `ens`, jusqu'à trouverun élément déjà présent dans cet ensemble, c'est-à-dire apparaissant déjà, plus tôt, dans le tableau.

Si nous souhaitons écrire une nouvelle version de cette fonction avec d'autres structures de données.

#### Deux approches rudimentaires
* Si nous souhaitons utiliser un tableau, en lieu et place d'un ensemble, cela va également fonctionner :

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = []
    for elt in tab:
        if elt in ens:
            return True
        ens.append(elt)
    return False

Le problème vient du test `elt in ens` dont le temps d'exécution est proportionnel au nombre d'élément stockés dans `ens`.  
On peut alors envisager une rechercher dichotomique mais cela implique aue le tableau soit trié et l'opération `append` ne serait alors plus adaptée (il faudrait insérer l'élément au bon endroit et décaler tous les suivants). Le coût serait donc alors porportionnel au nombre d'éléments du tableau.
* Une autre approche serait d'utiliser un grand tableau de booléens (un pour chaque date possible) et in décide que `ens[elt]` vaut `True` si on a enregistré `elt` dans `tab` et `False` sinon : 

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = [False] * 367
    for elt in tab:
        if ens[elt]:
            return True
        ens[elt] = True
    return False

Ceci permet d'obtenir un test qui a un coût minimal mais nous contraint à utiliser une structure de données "volumineuse" pour un tableau, à priori, petit.

#### Représentation plus compacte : le tableau de bits
Les booléens occupent, en effet, beaucoup d'espace en mémoire en Python.

In [ ]:
from sys import getsizeof

print("L'espace mémoire pour le booléen", True, "est", getsizeof(True), "octets")
print("L'espace mémoire pour le booléen", False, "est", getsizeof(False), "octets")

Un seul bit est réellement utile pour choisir entre les deux valeurs `True` ou `False`.  
On peut donc essayer d'utiliser des entiers représentant plusieurs booléeens et des opérations arithmétiques binaires.  
* Considérons un entier `ens` de 64 bits représentant en tableau `booleens` de 64 booléens, avec, comme clé de lecture que `booleens[elt]` vaut `True` si, et seulement si, le bit de rang `elt` de `ens` vaut `1`.  
Ainsi l'entier `26` qui s'écrit `00....0011010 ` représente le tableau de booléens `[False, True, False, True, True, True, False, False, ... , False]`, autrement dit l'ensemble $\{1,3,4\}$.  
* On peut alors utiliser l'entier $2^{elt}$, qui a tous ses bits à `0` sauf le bit de rang `elt` qui vaut `1`, comme révélateur.  
On calcule $2^{elt}$ avec l'opération de décalage `1 << x`.
* En utilisant un "ou binaire" : `ens | (1 << x)`, on fait passer le bit de rang `elt` de `ens` à `1` (et il y reste s'il y était déjà).  
En utilisant un "et binaire" : `ens & (1 << x)`, on obtient soit $2^{elt}$ si `booleens[elt]` vaut `True`, soit $0$ sinon.  

On obtient alors :

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = 0
    for elt in tab:
        if ens & (1 << x) != 0:
            return True
        ens = ens | (1 << x)
    return False

En Python, la taille des entiers est illimitée. En pensant à d'autres langages de programmation (pour lesquels la taille des entiers est limitée (à 64 bits, par exemple)), on pourrait utiliser un tableau d'entiers ordinaires.  
Il suffit alors de décomposer `elt` sous la forme `elt = i + 64j` avec les opérations de quotient et de reste de la division entière et on fait correspondre à `elt`le bit de rang `i` de l'entier d'indice `j`

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = [0] * 6
    for elt in tab:
        if ens[elt // 64] & (1 << (x % 64)) != 0:
            return True
        ens[elt // 64] = ens[elt // 64] | (1 << (x % 64))
    return False

Le code s'exécute aussi vite et on gagne un facteur 64 en occupation de la mémoire par rapport au tableau de booléens.  
Pour notre cas d'application, il suffit d'un tableau de six entiers pour représenter n'importe quel ensemble de dates d'anniversaires.

La représentation des ensembles développée dans les programmes précédents donne les idées essentielles de la structure de **tableau de bits**.  
Cette structure donne une représentation compacte d'un ensemble de booléens.  
Elle permet donc une meilleur utilisation des ressources de mémoire limitées comme la mémoire cache (mémoire accessible beaucoup plus rapidement que la mémoire vive de l'ordinateur et conservant temporairement les derniers éléments consultés).

#### Représentation plus souple : la table de hachage
Nous avons donc soit un "petit" tableau mais pour lequel chaque élément peut se trouver n'importe où, soit un "grand" tableau pour lequel il est immédiat de trouver la case du tableau liée à un élément donné. 

Nous pouvons essayer de combiner les avantages des deux méthodes : prenons un tableau de 23 cases (c'est le nombre d'éléments que l'on est susceptible de stocker dans notre ensemble) dans lequel chacune des 366 dates est liée, à priori, à une et une seule de ses cases.  
On peut alors enregistrer l'entier `elt` dans la case d'indice `elt % 23`.  
Il faut alors stocker, dans la case `ens[i]` un tableau contenant tous les éléments `elt` enregistrés tels que `elt % 23` vaut `i`.

In [ ]:
def contient_doublon(tab : list) -> bool:
    """Le tableau tab contient-il un doublon?"""
    ens = [[] for _ in range(23)]
    for elt in tab:
        if elt in ens[elt % 23]:
            return True
        ens[elt % 23].append(elt)
    return False

Il y a un maximum de 23 éléments à répartir entre 23 petits tableaux, chacun d'entre eux sera quasiment vide : le coût des tests `elt in ens[elt % 23]` sera ainsi, en général, négligeable.  
Les jours de naissance étant, à peu près, également probables, la répartition dans les différents paquets a de grandes chances d'être équitable.

#### Factorisation du code


## Sources :
* Balabonski Thibaut, et al. 2020. *Spécialité Numérique et sciences informatiques : 24 leçons avec exercices corrigés - Terminale - Nouveaux programmes*. Paris. Ellipse
* Interstices : [La naissance du génie logiciel](https://interstices.info/la-naissance-du-genie-logiciel/)
* Interstices : [Le hachage](https://interstices.info/le-hachage/)
* [Complexité temporelle en Python](https://wiki.python.org/moin/TimeComplexity)